#Linear regression

yo.

In [1]:
(require gamble 
         racket/class
         racket/list
         racket/vector
         "c3_helpers.rkt")

##Make up a "true" regression line

We pick a slope, a y-intercept, and an error sigma, which we will want to infer from some data.

In [2]:
(define true-slope 3)
(define true-intercept 12)
(define true-error-sigma 20)

##Make some data

Let's see how well we can do with 30 evenly-spaced data points.

In [3]:
(define n-points 30)
(define xs (build-vector n-points (lambda (x) x)))
(define ys (build-vector n-points
                         (lambda (x) (+ (* x true-slope) 
                                        true-intercept 
                                        (normal 0 true-error-sigma)))))

In [4]:
(scatter-c3 (vector->list xs) (vector->list ys))

(c3-data . #hasheq((data . #hasheq((type . scatter) (xs . #hasheq((ys1 . xs1))) (columns . ((xs1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29) (ys1 34.44910394378268 19.98471679616327 26.493814304883806 22.809431745043643 36.57633654003425 -10.2977859104084 15.397228483449684 36.286963392612236 42.231309707040545 46.872343818773 47.928846985026055 65.93344198809476 74.39901795908321 56.858494232865404 65.11754396921108 42.627828700222366 61.834255972350505 88.65696575648501 65.61503709145545 119.54332935731709 75.93347474650821 63.21809031387171 90.89013556242861 57.93961360837076 88.0309307346791 103.65815015001868 92.50755883975474 118.88514443891967 112.0773459213851 84.51854016283512))))) (axis . #hasheq((x . #hasheq((label . x))) (y . #hasheq((label . y)))))))

##Make a generative model inside a sampler

yessieree.

In [5]:
(define lr
  (mh-sampler
   (define A (normal 0 10))
   (define B (normal 0 10))
   (define E (gamma 1 15))
   (define (f x)
     (+ (* A x) B (normal 0 E)))
   (for ([i n-points])
     (observe (f i) (vector-ref ys i)))
   (vector A B E)))

In [6]:
(printf "Burn in:\n")
(for ([i 1000]) (void (lr))) ;; burn in
(define s1 (send lr info-in-struct))

(printf "\n1000 samples for MAP estimate:\n")
(send lr MAP-estimate 1000)
(send lr info)

;; This just gets summary stats.  I want to plot individual samples.
;;(sampler->statistics lr 1000)
;;(send lr info)

(printf "\n100 samples from posterior:\n")
(define smpls (generate-samples lr 100 #:thin 20))


Burn in:

1000 samples for MAP estimate:
== MH sampler
== Transition (single-site #:zone #f)
Total runs: 2000
Accepted traces: 447, 22.35%
Traces rejected by condition: 0, 0%
Traces rejected by MH threshold: 1553, 77.65%
Proposal perturbs: 2000
Fall-through perturbs: 0
-- Proposal (adaptive-drift)
Count: 2000
Scale increased: 2
Scale decreased: 30
Scale maintained: 15
-- Random selector

100 samples from posterior:


In [7]:
s1

#(struct:mh-info-struct 999 177 59/333 822 274/333 0 0 822 274/333)

In [7]:
(vector-ref smpls 0)


#(3.9652733066768713 5.324295034436617 20.47303068411699)

#Posterior distribution of regression lines

yo.

In [8]:
(define (xs-and-ys slope-intercept-error-samples x-min x-max)
   (list 
    (vector->list 
     (vector-map 
        (lambda (smp) (list x-min x-max))
        slope-intercept-error-samples))
    (vector->list
     (vector-map 
        (lambda (smp) (list (+ (vector-ref smp 1) (* x-min (vector-ref smp 0)))
                            (+ (vector-ref smp 1) (* x-max (vector-ref smp 0)))))
        slope-intercept-error-samples))))


In [9]:
(define end-pts (xs-and-ys (vector-take smpls 10) 0 30))


In [10]:
(multi-reg-line-plus-scatter-c3 
 (first end-pts)
 (second end-pts)
 (vector->list xs)
 (vector->list ys))

(c3-data . #hasheq((data . #hasheq((type . line) (types . #hasheq((true-ys . scatter))) (xs . #hasheq((true-ys . true-xs) (ys8 . xs8) (ys1 . xs1) (ys0 . xs0) (ys2 . xs2) (ys3 . xs3) (ys4 . xs4) (ys5 . xs5) (ys6 . xs6) (ys7 . xs7) (ys9 . xs9))) (columns . ((true-xs 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29) (true-ys 34.44910394378268 19.98471679616327 26.493814304883806 22.809431745043643 36.57633654003425 -10.2977859104084 15.397228483449684 36.286963392612236 42.231309707040545 46.872343818773 47.928846985026055 65.93344198809476 74.39901795908321 56.858494232865404 65.11754396921108 42.627828700222366 61.834255972350505 88.65696575648501 65.61503709145545 119.54332935731709 75.93347474650821 63.21809031387171 90.89013556242861 57.93961360837076 88.0309307346791 103.65815015001868 92.50755883975474 118.88514443891967 112.0773459213851 84.51854016283512) (xs0 0 30) (xs1 0 30) (xs2 0 30) (xs3 0 30) (xs4 0 30) (xs5 0 30) (xs6 0 30) (xs7 0 30) (xs8 0 